In [ ]:
#Configuration section
#Change parameters below if you want to export results t Excel workbooks in a blob storage account

#Account name of the blob stroage account the workbooks are saved to
blob_account_name='<YourAccountName>'

#Account key of the blob stroage account the workbooks are saved to
blob_account_key='<YourAccountKey>'

#Blob container name taht the excel workbooks will be saved to
blobContainerName='<YourContainerName>'

#By default use the same cosmos account in which the notebook is stored in
client = cosmos_client
#Alternatively you could point to another cosmso account via connection string for example
#client = CosmosClient.from_connection_string('<YourConnectionString>')

In [ ]:
#Installing modules needed to export result to Excel workbooks on Blob storage
!pip install XlsxWriter --user
!pip install azure.storage --user

In [ ]:
#Running PartitionKeyStatistics
import azure.cosmos
import datetime
import io
import pandas as pd
from azure.cosmos import CosmosClient
from azure.storage.blob import (BlockBlobService)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

for dbProperties in client.list_databases():
    database = client.get_database_client(dbProperties['id'])
    
    for collectionProperties in database.list_containers():
        container = database.get_container_client(collectionProperties['id'])

        containerProperties = container.read(populate_partition_key_range_statistics = True, populate_quota_info = True)

        partitionQuotas = {
            'PartitionId': [],
            'DocumentCount': [],
            'StorageSize': [],
        }

        partitionKeyStatistics = {
            'PartitionId': [],
            'LogicalPartitionKeyValue': [],
            'StorageSize': [],
        }

        for statisticsRecord in containerProperties['statistics']:
            partitionId = str(statisticsRecord['id'])
            partitionQuotas['PartitionId'].append(partitionId)
            partitionQuotas['DocumentCount'].append(float(statisticsRecord['documentCount']))
            partitionQuotas['StorageSize'].append(float(statisticsRecord['sizeInKB']))

            for partitionKeyStatisticsRecord in statisticsRecord['partitionKeys']:
                partitionKeyStatistics['PartitionId'].append(partitionId)
                partitionKeyStatistics['LogicalPartitionKeyValue'].append(partitionKeyStatisticsRecord['partitionKey'])
                partitionKeyStatistics['StorageSize'].append(float(partitionKeyStatisticsRecord['sizeInKB']))

        partitionQuotaDataFrame = pd.DataFrame(partitionQuotas) 
        partitionKeyStatisticsDataFrame = pd.DataFrame(partitionKeyStatistics) 
        
        print()
        print('Database {0:s} - Container {1:s}'.format(dbProperties['id'], collectionProperties['id']))
        print('QUOTA')
        print(partitionQuotaDataFrame)
        
        print()
        print('PARTITION KEY STATISTICS')
        print(partitionKeyStatisticsDataFrame)
        
        #Storing statistics for each ocntainer in an Ecel workbook
        filename = '{0:s}_{1:s}_{2:s}.xlsx'.format(dbProperties['id'], collectionProperties['id'], datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S'))
        output= io.BytesIO()
        writer = pd.ExcelWriter(output, engine='xlsxwriter', options={'in_memory': True})
        
        partitionQuotaDataFrame.to_excel(writer, sheet_name='Quota', index=False)
        partitionKeyStatisticsDataFrame.to_excel(writer, sheet_name='Partition key statistics', index=False)
        writer.save()

        output.seek(0)
        blobService = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
        
        blobService.create_blob_from_bytes(blob_container_name, filename, output.read())
        print()
        print('Exported to file: {0:s}'.format(filename))